In [1]:
import pytfa

from optlang.exceptions import SolverError

from cobra.flux_analysis import flux_variability_analysis
from cobra.io import load_matlab_model, load_json_model


from pytfa.io import import_matlab_model, load_thermoDB,                    \
                            read_lexicon, annotate_from_lexicon,            \
                            read_compartment_data, apply_compartment_data


ImportError: cannot import name 'save_json_model' from 'pytfa.io' (/Users/ssulheim/opt/anaconda3/envs/pytfa/lib/python3.10/site-packages/pytfa/io/__init__.py)

In [2]:
from pathlib import Path
import cobra

# Build iJO1366 TFA model

In [3]:
solver = 'optlang_gurobi'
pytfa_folder = Path("/Users/ssulheim/git/pytfa")
thermo_data = load_thermoDB(pytfa_folder / "data" / "thermo_data.thermodb")

In [4]:
 # We import pre-compiled data as it is faster for bigger models
cobra_model = cobra.io.read_sbml_model('../../models/e_coli/iJO1366.xml')
# The lexicon is just a csv file with seed annotat
# lexicon = read_lexicon(pytfa_folder / 'models/iJO1366/lexicon.csv')
# annotations = lexicon.to_dict(orient = 'index')
# the mytfa seems to fix the seed annotations directly from the xml model

In [5]:
compartment_data = read_compartment_data(str(pytfa_folder / 'models/iJO1366/compartment_data.json'))



In [6]:
# Initialize the cobra_model
mytfa = pytfa.ThermoModel(thermo_data, cobra_model)


Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmpxcmgu6g1.lp
Reading time = 0.01 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


2023-03-03 11:03:23,695 - thermomodel_None - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


In [7]:
# Just to check
lexicon = read_lexicon(pytfa_folder / 'models/iJO1366/lexicon.csv')
annotations = lexicon.to_dict(orient = 'index')
mytfa.metabolites.ac_c.annotation
annotations['ac_c']

{'seed_id': 'cpd00029'}

In [10]:


# Annotate the cobra_model
# annotate_from_lexicon(mytfa, lexicon) don't need this
apply_compartment_data(mytfa, compartment_data)

biomass_rxn = 'BIOMASS_Ec_iJO1366_WT_53p95M'#'Ec_biomass_iJO1366_WT_53p95M'

In [11]:
mytfa.name = 'leakage'
mytfa.solver = solver
mytfa.objective = biomass_rxn
mytfa.description = "Model used for predicting shadow prices"

In [12]:
def apply_solver_settings(model, solver = solver):
    model.solver = solver
    # model.solver.configuration.verbosity = 1
    model.solver.configuration.tolerances.feasibility = 1e-9
    if solver == 'optlang_gurobi':
        model.solver.problem.Params.NumericFocus = 3
    model.solver.configuration.presolve = True

apply_solver_settings(mytfa)

In [13]:
## FBA
fba_solution = cobra_model.optimize()
fba_value = fba_solution.objective_value
# fva = flux_variability_analysis(mytfa)

In [14]:
## TFA conversion
mytfa.prepare()
mytfa.convert()#add_displacement = True)

2023-03-03 11:05:16,740 - thermomodel_None - INFO - # Model preparation starting...
2023-03-03 11:05:17,924 - thermomodel_None - INFO - # Model preparation done.
2023-03-03 11:05:17,925 - thermomodel_None - INFO - # Model conversion starting...
2023-03-03 11:05:24,301 - thermomodel_None - INFO - # Model conversion done.
2023-03-03 11:05:24,302 - thermomodel_None - INFO - # Updating cobra_model variables...
2023-03-03 11:05:24,349 - thermomodel_None - INFO - # cobra_model variables are up-to-date


In [15]:
## Info on the cobra_model
mytfa.print_info()


                                                   value
key                                                     
name                                             leakage
description      Model used for predicting shadow prices
num constraints                                     9554
num variables                                      12137
num metabolites                                     1805
num reactions                                       2583
                          value
key                            
num metabolites(thermo)  1805.0
num reactions(thermo)       0.0
pct metabolites(thermo)   100.0
pct reactions(thermo)       0.0


In [16]:
## Optimality
tfa_solution = mytfa.optimize()
tfa_value = tfa_solution.objective_value

In [17]:
tfa_value

0.9865144469529776

In [18]:
if tfa_value < 0.1:
    from pytfa.optim.relaxation import relax_dgo

    mytfa.reactions.get_by_id(biomass_rxn).lower_bound = 0.5*fba_value
    relaxed_model, slack_model, relax_table = relax_dgo(mytfa)

    original_model, mytfa = mytfa, relaxed_model

    print('Relaxation: ')
    print(relax_table)
    
    tfa_solution = mytfa.optimize()
    tfa_value = tfa_solution.objective_value

In [19]:
tfa_value

0.9865144469529776

In [20]:
tfa_solution = mytfa.optimize()

# Store model

In [36]:
def save_json_model(model, filepath):
    # filepath = check_json_extension(filepath)
    obj = pytfa.io.dict.model_to_dict(model)
    with open(filepath, 'w') as fid:
        json.dump(obj, fid, cls=pytfa.io.MyEncoder)

In [37]:
tfa_path = '../../models/e_coli/iJO1366_tfa.json'
save_json_model(mytfa, tfa_path)

# Load model

In [38]:
def load_json_model(filepath):
    with open(filepath, 'r') as fid:
        obj = json.load(fid)

    model = pytfa.io.dict.model_from_dict(obj)
    return model


# Predict shadow prices

In [41]:
m_ids = [m.id for m in mytfa.metabolites if m.compartment == 'c']    

In [45]:
shadow_prices = {}
for m_id in m_ids:
    with mytfa as model:
        m = model.metabolites.get_by_id(m_id)
        try:
            r = model.add_boundary(m, type = "demand")
        except ValueError:
            r = model.reactions.get_by_id('DM_{0}'.format(m_id))
        
        r.bounds = (0.01, 0.01)
        solution = model.slim_optimize()
    shadow_prices[m_id] = (solution-tfa_value)/0.01

(0, 1000.0)